In [1]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")
import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response



In [2]:
num_rows_batch_publish=5000 # number of objects to be batch published to Scorpio

# Notes:
Consider every hexagon as an entity, and sensor data as properties

In [3]:
import pandas as pd
import geopandas as gpd



In [4]:
# import building datasets
gdf_madrid = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-ESP_8_1_3_10_1-Madrid.gpkg')
gdf_berlin = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-DEU_3_1_1_1_1-Berlin.gpkg')
gdf_warszawa = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-POL_7_36_1-Warszawa.gpkg')
gdf_vichy = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-FRA_1_2_3_11_1-Vichy.gpkg')
gdf_amsterdam = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-NLD_9_4_1-Amsterdam.gpkg')
gdf_basel = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-CHE_5_1_1-Basel.gpkg')
gdf_brugge = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-BEL_2_5_1_3_1-Brugge.gpkg')

gdf_amsterdam

,id,height,age,type,id_source,type_source,geometry
0,v0.1-NLD.9.4_1-0,8.810910,1923,NaN,5603857,NaN,"POLYGON ((3973644.962 3268203.243, 3973644.962..."
1,v0.1-NLD.9.4_1-1,7.467500,1950,NaN,17627787,NaN,"POLYGON ((3976770.186 3265556.206, 3976770.185..."
2,v0.1-NLD.9.4_1-2,8.063870,1931,NaN,5605121,NaN,"POLYGON ((3975898.760 3265769.202, 3975898.761..."
3,v0.1-NLD.9.4_1-3,13.437600,1880,NaN,14570411,NaN,"POLYGON ((3973594.069 3262404.139, 3973594.069..."
4,v0.1-NLD.9.4_1-4,5.591080,1988,NaN,12864206,NaN,"POLYGON ((3973893.366 3267633.403, 3973893.367..."
...,...,...,...,...,...,...,...
169188,v0.1-NLD.9.4_1-169188,14.407900,1921,NaN,2742030,NaN,"POLYGON ((3971342.203 3261783.586, 3971342.203..."
169189,v0.1-NLD.9.4_1-169189,16.013300,1913,NaN,3254518,NaN,"POLYGON ((3971113.150 3261436.177, 3971113.149..."
169190,v0.1-NLD.9.4_1-169190,2.777260,1981,NaN,4025546,NaN,"POLYGON ((3967075.939 3263069.039, 3967075.938..."
169191,v0.1-NLD.9.4_1-169191,14.290900,1965,NaN,10711828,NaN,"POLYGON ((3966574.704 3263125.618, 3966574.704..."


In [ ]:

madrid_hexagons_gdf = pd.read_pickle('./../cgarrido/madrid_data/hex_grid/hexagons.pkl')
berlin_hexagons_gdf=  pd.read_pickle('./../cgarrido/berlin_data/hex_grid/hexagons.pkl')
berlin_hexagons_gdf

In [ ]:
madrid_hexagons_gdf

In [8]:


gdf_madrid= gdf_madrid.to_crs(epsg=3857)
gdf_berlin= gdf_berlin.to_crs(epsg=3857)

gdf_madrid_mapped_buildings = gpd.sjoin(gdf_madrid, madrid_hexagons_gdf).drop(columns=["index_right"])

gdf_berlin_mapped_buildings = gpd.sjoin(gdf_berlin, berlin_hexagons_gdf).drop(columns=["index_right"])

gdf_berlin_mapped_buildings


,id,height,age,type,id_source,type_source,geometry,hexagon_id,hexagon_x,hexagon_y
19259,v0.1-DEU.3.1.1.1_1-19259,21.57,NaN,NaN,BLDG_0003000e0013a1cc,1211.0,"POLYGON ((1497558.100 6889109.682, 1497554.398...",14,-2,0
19261,v0.1-DEU.3.1.1.1_1-19261,6.92,NaN,NaN,BLDG_0003000e001396be,1137.0,"POLYGON ((1497315.376 6889005.364, 1497315.375...",14,-2,0
19262,v0.1-DEU.3.1.1.1_1-19262,7.89,NaN,NaN,BLDG_000300000063aef8,1149.0,"POLYGON ((1497476.365 6889127.229, 1497476.365...",14,-2,0
19264,v0.1-DEU.3.1.1.1_1-19264,17.03,NaN,NaN,BLDG_0003000e0013a1c8,1211.0,"POLYGON ((1497361.357 6889111.827, 1497361.356...",14,-2,0
19265,v0.1-DEU.3.1.1.1_1-19265,22.24,NaN,NaN,BLDG_0003000e001396b6,1231.0,"POLYGON ((1497507.411 6889007.697, 1497507.409...",14,-2,0
...,...,...,...,...,...,...,...,...,...,...
538438,v0.1-DEU.3.1.1.1_1-538438,0.93,NaN,NaN,DEB_LOD2_UUID_2b05c943-0a28-4567-89c9-e652996d...,NaN,"POLYGON ((1485916.101 6910904.824, 1485916.100...",70,4,-1
538449,v0.1-DEU.3.1.1.1_1-538449,1.54,NaN,NaN,DEB_LOD2_UUID_c1ef21de-50ca-4d37-8118-33e4f456...,NaN,"POLYGON ((1486106.968 6910788.338, 1486106.966...",70,4,-1
538454,v0.1-DEU.3.1.1.1_1-538454,2.49,NaN,NaN,DEB_LOD2_UUID_c2fe762c-e122-40d9-9866-7102e0df...,NaN,"POLYGON ((1485910.027 6910886.942, 1485910.027...",70,4,-1
538460,v0.1-DEU.3.1.1.1_1-538460,2.88,NaN,NaN,DEB_LOD2_UUID_f73dce9d-8b92-4b65-89ee-ede0a978...,NaN,"POLYGON ((1484739.903 6909988.569, 1484739.903...",70,4,-1


In [9]:
gdf_list= [gdf_madrid_mapped_buildings, gdf_berlin_mapped_buildings]
entity_id_list = [ "urn:ngsi-ld:Building:building-Madrid", "urn:ngsi-ld:Building:building-Berlin"]
entity_type_list = ['Building'] * len(gdf_list)


In [10]:
import time

example_flag=False
for entity_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True

    hexagon_id_list = tmp_gdf[['hexagon_id']].drop_duplicates().values.tolist()
    print("Info: Unique hexagons for the dataset : "+ str(hexagon_id_list))
    feature_list= tmp_gdf.columns.tolist()
    dataset_length = len(tmp_gdf)
    for hexagon_id in hexagon_id_list:
        hexagon_id = hexagon_id[0]
        hexagon_gdf = tmp_gdf[tmp_gdf['hexagon_id']== hexagon_id]

        counter =0
        for index, row in hexagon_gdf.iterrows():
            if counter %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
                entity_obj = Entity(entity_type_list[entity_index], entity_id_list[entity_index] + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity
            for f in range(len(feature_list)):
                if feature_list[f] =='type': feature_list[f] ='building_type' # quick fix for the error of "type" conflicting with ngsi-ld "type"
                if(feature_list[f] not in ['datetime','type']):
                    entity_obj.add_prop(feature_list[f], str(row[f]))                 
            if example_flag:
                print('INFO: Example data from the dataset to be published - as an example on how data looks.')
                print(entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(hexagon_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(entity_obj.to_dict()) # publish the entity
            counter+=1 
        print("... published data for hexagon: " +  str(hexagon_id))

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully with entity ID: ' + str(entity_id_list[entity_index]) 
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))


Info: Unique hexagons for the dataset : [[56], [57], [73], [50], [60], [58], [11], [18], [77], [62], [26], [63], [30], [78], [13], [52], [44], [81], [43], [66], [80], [46], [6], [16], [67], [24], [5], [39], [53], [37], [21], [65], [41], [48], [42], [22], [69], [20], [23], [61], [14], [4], [71], [40], [70], [38], [64], [28], [0], [25], [79], [75], [12], [49], [3], [90], [76], [84], [45], [47], [89], [55], [15], [2], [29], [1], [7], [10], [87], [59], [9], [34], [36], [31], [35], [17], [19], [8], [32], [54], [27], [33], [85], [83], [68]]
INFO: Example data from the dataset to be published - as an example on how data looks.
{'id': {'type': 'Property', 'value': 'v0.1-ESP.8.1.3.10_1-111'}, 'type': 'Building', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'height': {'type': 'Property', 'value': '2.5'}, 'age': {'type': 'Property', 'value': '2000.0'}, 'building_type': {'type': 'Property', 'value': 'non-residential'}, 'id_source': {'type': 'Property', 'value': 'ES.